In [1]:
import gym
import numpy as np
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count

import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from gym import error, spaces, utils
from gym.utils import seeding

# Create a custom environment 
class WTAEnv(gym.Env):
    metadata = {'render.modes': ['console']}

    def __init__(self, assignment, target_values, prob, device):
        ''' 
        Assignment - 1D array that maps weapons to assigned target
        target_values - 1D array that maps targets to their value when destroyed
        prob - m x n array where prob[i, j] = probability of weapon i killing target j
        ''' 
        super(WTAEnv, self).__init__()
        
        self.n = len(target_values)
        self.m = len(assignment)

        self.start_assignment = assignment
        self.assignment = assignment
        
        # one hot encoded version of assignment, for quick reward computation
        self.assignment_onehot = self.get_onehot_assignments()
        
        self.target_values = target_values 
        self.prob = prob
        # q = probability array of survival
        self.q = 1 - self.prob
        
        # Current expected value of the assignment
        self.value = self.assignment_value()
        
        # The action space - a number 0 <= i < m * n, where
        # (weapon = i // n, target = i % n)
        # Assigns weapon to target (one target per weapon)
        self.action_space = spaces.MultiDiscrete([self.m * self.n])
        
        self.device = device
        
    def decode_action(self, action):
        '''
        Given an action, return the weapon and target associated with
        that action.
        '''
        return action // self.n, action % self.n
        
    def get_onehot_assignments(self):
        '''
        Get a one-hot encoded representation of the assignments.
        This is used only because this representation is convenient and
        fast for computing the assignment value, since we can use
        vectorization and not use for loops. Not sure how much this
        actually helps though.
        '''
        onehot = np.zeros((self.m, self.n))
        onehot[np.arange(onehot.shape[0]), self.assignment] = 1
        return onehot
 
    def assignment_value(self):
        '''
        Compute the expected value of our assignment.
        E = Sum over targets i [P(target i killed) * Value(i)]
        where P(target i killed) = 1 - P(i survives)
        where P(i surves) = 1 - Product over weapons j [P(i survives j) = q[i, j]]
        '''
        pkill = 1 - np.prod(self.q ** self.assignment_onehot, axis=0)
        expected_value = np.dot(pkill, self.target_values)
        return expected_value
        
    def step(self, action):
        '''Perform action on the current state'''
        weapon, target = self.decode_action(action[0])
        if weapon < 0 or weapon >= self.m or target < 0 or target >= self.n:
            raise ValueError("Received invalid action={} which is not part of the action space".format(action))
        # Update assignments
        old_target = self.assignment[weapon]
        self.assignment[weapon] = target
        self.assignment_onehot[weapon, old_target] = 0
        self.assignment_onehot[weapon, target] = 1

        # Reward is change in assignment value
        new_value = self.assignment_value()
        reward = new_value - self.value
        self.value = new_value

        # There is no stopping condition other than the max number of iterations
        done = False

        return (self.get_state(), reward, done, {})

    def get_state(self):
        '''
        Our state is a size m + n array.
        state[:m] is self.assignments
        state[m:] is self.target_values
        '''
        state = np.concatenate([self.assignment, self.target_values])
        return torch.tensor(state, device=self.device)

    def reset(self):
        '''
        Important: the observation must be a numpy array
        :return: (np.array) 
        '''
        self.assignment = self.start_assignment
        self.assignment_onehot = self.get_onehot_assignments()
        return self.get_state()

    def render(self, mode='human', close=False):
        pass

In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
def generate_initial_assignment(n, m):
    '''   
    Randomly assign weapons to targets
    ''' 
    assignment = np.random.randint(n, size=m)
    return assignment

In [5]:
steps_done = 0

def is_possible(state, weapon, target):
    '''
    We don't want to assign a weapon to the target if it is already assigned
    to the target, since this does not change the state at all.
    '''
    curr_target = state[weapon].item()
    return curr_target != target 

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        ## Exploit
        with torch.no_grad():
            policy_net.eval()
            state_batch = torch.unsqueeze(state, 1).transpose(0, 1).float()
            largest = torch.sort(policy_net(state_batch), descending=True, dim=1)[1]
            policy_net.train()
            
            # Try until we get a valid action
            for i in largest[0]:
                weapon = i / n
                target = i % n
                
                if is_possible(state, weapon.item(), target.item()):
                    return torch.tensor([i], device=device)
                
            # This should never happen
            raise ValueError('Invalid state: no possible action')
    else:
        ## Explore
        weapon = np.random.randint(m)
        curr_target = state[weapon].item()
        target = np.random.randint(n)
        while target == curr_target:
            target = np.random.randint(n)
        
        action = weapon * n + target
        return torch.tensor([action], device=device, dtype=torch.long)

episode_durations = []

def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

In [75]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
        
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.stack([s for s in batch.next_state
                                                if s is not None])
        
    state_batch = torch.stack(batch.state)
    action_batch = torch.stack(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
#     print(next_state_values[non_final_mask].shape)
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values.double(), expected_state_action_values.unsqueeze(1).double())
    value_loss = loss.item()
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()
    
    return value_loss

In [76]:
'''Take in n by m matrix, convert it to 1D feature vector '''
class DQN(nn.Module):
    def __init__(self, n, m, embedding_size=8):
        super(DQN, self).__init__()
        # The assignment becomes embedded, so it has size m * embedding_size
        # when flattened
        # The n comes from the values attached
        self.assignment_size = m * embedding_size
        self.input_size = self.assignment_size + n
        self.output_size = m * n
        self.n = n
        self.m = m
        
        units = 50

        self.embedding_size = embedding_size
        # Embed the targets, since the actual numerical value of the
        # targets don't mean anything
        # Another idea: skip the middleman and replace the targets
        # with the target values
        self.embedding = nn.Embedding(n, self.embedding_size)
        self.lin1 = nn.Linear(self.input_size, units)
        self.drop1 = nn.Dropout(0.2)
        self.lin2 = nn.Linear(units, self.output_size)
        self.drop2 = nn.Dropout(0.2)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, state):
        assignment = state[:, :self.m].long()
        assignment = self.embedding(assignment)
        
        values = state[:, self.m:].float()
                
        # Flatten the assignment embedding
        assignment = assignment.view(-1, self.assignment_size).float() 
        
        # and concatenate the values
        x = torch.cat([assignment, values], dim=1)
        
        x = F.relu(self.drop1(self.lin1(x)))
        x = F.relu(self.lin2(x))
        return x

In [101]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 50
MAX_ITERATIONS = 500

# n - number of targets
n = 4
# m - number of weapons
m = 5
assert n > 1

lower_val = 25
upper_val = 50
lower_prob = 0.6
upper_prob = 0.9
values = np.random.uniform(lower_val, upper_val, n)
prob = np.random.uniform(lower_prob, upper_prob, (m, n))
assignment = generate_initial_assignment(n, m)
env = WTAEnv(assignment, values, prob, device)

policy_net = DQN(n, m, n // 2).to(device)
target_net = DQN(n, m, n // 2).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

In [102]:
num_episodes = 15
env.reset()
init_state = env.get_state() 
for i_episode in range(num_episodes):
    # Initialize the environment and state
    env.reset()
    state = init_state
    for t in range(MAX_ITERATIONS):
        print(f'episode {i_episode}/{num_episodes}, iteration {t}/{MAX_ITERATIONS}', end=' ')
        # Select and perform an action
        action = select_action(state)
        observation, reward, done, _ = env.step(action)
        reward = torch.tensor([reward], device=device)

        if not done:
            next_state = observation
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        loss = optimize_model()
        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break
        
        print(f'loss: {loss}', end='\r')
    print()
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print()
print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

episode 0/15, iteration 499/500 loss: 2.0261524435764686
episode 1/15, iteration 499/500 loss: 4.9050608593957575
episode 2/15, iteration 499/500 loss: 4.3939150354779835loss: 4.528499014440507
episode 3/15, iteration 499/500 loss: 4.1426145519815413loss: 3.774118627185442
episode 4/15, iteration 499/500 loss: 3.4468548120016154 loss: 3.5652665707681415
episode 5/15, iteration 499/500 loss: 3.3691515881484926
episode 6/15, iteration 499/500 loss: 2.6290018776910175
episode 7/15, iteration 499/500 loss: 2.5240078030234194
episode 8/15, iteration 499/500 loss: 1.6320195614823816
episode 9/15, iteration 499/500 loss: 2.4951575539090225
episode 10/15, iteration 499/500 loss: 1.8746683249655045
episode 11/15, iteration 499/500 loss: 1.3916716545846242oss: 1.920757652821843loss: 1.51725985710407
episode 12/15, iteration 499/500 loss: 1.4885495817520358
episode 13/15, iteration 499/500 loss: 1.1838817377868453
episode 14/15, iteration 499/500 loss: 0.9955027656697102ss: 1.702412812951216loss:

In [145]:
policy_net.eval()

DQN(
  (embedding): Embedding(4, 2)
  (lin1): Linear(in_features=14, out_features=50, bias=True)
  (drop1): Dropout(p=0.2, inplace=False)
  (lin2): Linear(in_features=50, out_features=20, bias=True)
  (drop2): Dropout(p=0.2, inplace=False)
)

In [146]:
assignment = generate_initial_assignment(n, m)

In [147]:
test_env = WTAEnv(assignment, values, prob, device)

In [148]:
test_env.reset()

tensor([ 3.0000,  3.0000,  3.0000,  0.0000,  0.0000, 31.9710, 34.0953, 27.0945,
        29.3957], device='cuda:0', dtype=torch.float64)

In [149]:
test_env.value

59.479090569076405

In [150]:
state = test_env.get_state().unsqueeze(1).transpose(0, 1)
best_action = policy_net(state).max(1)[1]

a = best_action.item()
print(a // n, a % n)

0 1


In [151]:
test_env.step(best_action)

(tensor([ 1.0000,  3.0000,  3.0000,  0.0000,  0.0000, 31.9710, 34.0953, 27.0945,
         29.3957], device='cuda:0', dtype=torch.float64),
 21.5414138235122,
 False,
 {})

In [152]:
test_env.value

81.0205043925886

In [129]:
policy_net(state).reshape(5, 4)

tensor([[ 0.0000, 37.2045,  0.0000,  0.0000],
        [10.0700, 13.8417, 24.7925, 13.4367],
        [ 0.0000, 28.7972, 22.6149, 24.7226],
        [12.0092,  0.0000, 14.5318, 22.7798],
        [33.5026,  0.0000,  0.0000,  8.3360]], device='cuda:0',
       grad_fn=<ViewBackward>)